INSTRUCTIONS

1. open file in Jupyter Notebook
2. put (updated) excel sheet in script directory and store name of file in "excel_sheet" variable
3. install packages in terminal ('pip install pandas plotly')
4. store list of desired items in "items" variable
5. run displayDeliveries(array)

In [16]:
# excel_input = 'Romania 2017-2022.xlsx'
excel_input = 'output.xlsx'
items = ['ANELLO TEN INTERNO 45X62X8 NBR','KIT SEAL CRANE D16 21-10K55']

In [2]:
import pandas as pd
import plotly.graph_objects as go

In [8]:
df = pd.read_excel(excel_input)

In [29]:
def getDeliveries(items):
    queryStr = ' or '.join(["ItemName == '{item}'".format(item=item) for item in items])
    deliveries = df.query(queryStr)[['ItemName','Del.Addr','OrdQty','City','lat','lng']].reset_index()

    for row in range(deliveries.shape[0]):
        delivery = deliveries.iloc()[row]

        deliveries.at[row,'text'] = 'Item: {item}<br>City: {city}<br>Volume: {qty}'.format(
                            item=delivery['ItemName'],
                            city=delivery['City'],
                            qty=delivery['OrdQty'])
    return deliveries

def getGeoObj(df,item,index):
    minQty = min(df['OrdQty'].tolist()) if df.shape[0] else 0
    maxQty = max(df['OrdQty'].tolist()) if df.shape[0] else 0

    deliveries = df.query("ItemName == '{item}'".format(item=item))
        
    geoObj = go.Scattergeo(
        lon = deliveries['lng'],
        lat = deliveries['lat'],
        text = deliveries['text'],
        mode = 'markers',
        marker = dict(
            color = deliveries['OrdQty'],
            colorscale = ['red','blue'],
            cauto = False,
            autocolorscale = False,
            cmin = minQty,
            cmax = maxQty,
            colorbar = None if index > 0 else dict(
                title = 'Volume',
                titleside = 'top',
                tickmode = 'array',
            )
        )
    )

    geoObj['marker_symbol'] = symbols[index%len(symbols)]
    geoObj['name'] = item

    return geoObj

def displayDeliveries(items):
    deliveries = getDeliveries(items)

    fig = go.Figure(data=[getGeoObj(deliveries,item,index) for index,item in enumerate(items)])

    fig.update_geos(
        lataxis_range=[43, 49],
        lonaxis_range=[20, 30],
    )

    title = 'Delivery Locations for '
    title += '{amount} items'.format(amount=len(items)) if len(items) > 1 else items[0]

    fig.update_layout(
        title=title,
        geo_scope='europe',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=-1
        )
    )

    fig.show()

In [17]:
# items = list(set(df['ItemName'].head(100)))
symbols = ['circle','square','diamond','cross','x','triangle-up',
    'triangle-down','triangle-left','triangle-right']

In [32]:
displayDeliveries(items)